In [3]:
import pandas as pd
import numpy as np
from math import log2
data=pd.read_csv("id3_example.csv")
data

,a1,a2,a3,target
0,T,hot,high,no
1,T,hot,high,no
2,F,hot,high,yes
3,F,cool,normal,yes
4,F,cool,normal,yes
5,T,cool,high,no
6,T,hot,high,no
7,T,hot,normal,yes
8,F,cool,normal,yes
9,F,cool,high,yes


In [100]:
class node:
    def __init__(self,column_name,columns_values) -> None:
        self.name=column_name
        self.columns_values=columns_values
        self.pointers={j:None for j in columns_values}
    
    def update_pointers(self,next_nodes):
        for j,node in zip(self.columns_values,next_nodes):
            self.pointers[j]=node


    def forward(self,record,columns):
            for i,(column,data) in enumerate(zip(columns,record)):
                if(self.name==column):
                    columns.remove(self.name)
                    if(isinstance(self.pointers[data],node)):
                        return self.pointers[data].forward(record.drop(self.name),columns)
                    else:
                        return self.pointers[data]
                    

class ID3:
    def __init__(self,data,target_column) -> None:
        self.data=data
        self.target_column=target_column
        self.each_columns_unique_value={i:data[i].unique() for i in self.data.columns}
        self.target_column_values=data[target_column].unique()
        self.features=data.drop(columns=[target_column])
        self.targets=data[target_column]
        self.total=len(self.features)
        self.root=None
    def entropy(self,values,total_num_of_values):
        individual_entropies=[]
        for value in values:
            #print(value)
            if(value!=0):
                individual_entropies.append((-value/total_num_of_values)*log2(value/total_num_of_values))
            else:
                individual_entropies.append(0)
        return sum(individual_entropies)

             
    def get_info_gain(self,nodes=None,node_values=None):
        subset_data=self.data
        attribute_value_dict={}
        if nodes!=None:
            for node,value in zip(nodes,node_values):
                subset_data=subset_data[subset_data[node]==value]
            
        #print(subset_data)
        values=subset_data[self.target_column].value_counts() 
        main_entropy_value=self.entropy(values,sum(list(values)))

        """ for i in columns_to_use:
            attribute_value_dict[i]={j:{k:0 for k in self.target_column_values} for j in self.each_columns_unique_value[i]}
            for i in range(len(subset_data)):
                row=subset_data[[i,self.target_column]].iloc[i]
                attribute_value_dict[i][row[i]][row[self.target_column]]+=1
            entropies={j:self.entropy(attribute_value_dict[i][j].values,(sum(attribute_value_dict[i][j].values))) for j in self.each_columns_unique_value}
         """
        columns=list(self.data.columns)
        columns.remove(self.target_column)
        entropies={}
        if nodes!=None:
            for i in nodes:
                columns.remove(i)
        if(len(columns)==0):
             return (subset_data.iloc[0][self.target_column],)
        
        for i in columns:
            sum_of_entropies=0
            for j in subset_data[i].unique():
                #print(subset_data[subset_data[i]==j])
                values=list(subset_data[subset_data[i]==j][self.target_column].value_counts())
                #print(sum(values))
                #print("entropy values: ",self.entropy(values,sum(values)))
                sub_entropy=(len(subset_data[subset_data[i]==j])/len(subset_data))*self.entropy(values,sum(values))
                sum_of_entropies+=sub_entropy
            entropies[i]=main_entropy_value-sum_of_entropies
        #print(entropies)
        max=-10**10
        max_col=''
        for i in entropies.keys():
            if(max<entropies[i]):
                max=entropies[i]
                max_col=i
        return max_col
    
    def recursive_tree_building(self,nodes,nodes_values,set_values):
        #print("cur_node: ",nodes[-1])
        cur_node=node(nodes[-1],list(self.data[nodes[-1]].unique()))
        next_nodes=[]
        for value in nodes_values:
            best_col=self.get_info_gain(nodes=nodes,node_values=set_values+[value])
            #print("best columns in recursion: ",best_col)
            if(isinstance(best_col,tuple)):
                   return best_col[0]
            nodes.append(best_col)
            set_values.append(value)
            #print(nodes,set_values)
            next_nodes.append(self.recursive_tree_building(nodes,list(self.data[best_col].unique()),set_values))
            nodes.remove(best_col)
            set_values.remove(value)
        cur_node.update_pointers(next_nodes)
        return cur_node
              

    def fit(self):
        node=self.get_info_gain()
        #print(node)
        nodes=[node]
        nodes_values=list(self.data[node].unique())
        self.root=self.recursive_tree_building(nodes,nodes_values,[])
    
    def infer(self,record):
        #print(record)
        columns=list(self.data.columns)
        columns.remove(self.target_column)
        print("inference is: ",self.root.forward(record,columns))



In [108]:

algo=ID3(data,'target')
algo.fit()
algo.infer(data.drop(columns=['target']).iloc[9])


inference is:  yes


In [109]:
for i in enumerate({'a':12,'b':123}):
    print(i)

(0, 'a')
(1, 'b')


In [110]:
type(data[data['a1']=='T'])

pandas.core.frame.DataFrame

In [111]:
for i in enumerate(zip(data[data['a1']=='T'].iloc[1],data.columns)):
    print(i)

(0, ('T', 'a1'))
(1, ('hot', 'a2'))
(2, ('high', 'a3'))
(3, ('no', 'target'))


In [112]:
k=list(data.columns)
k.remove('a1')
k

['a2', 'a3', 'target']

In [113]:
k=type([1,2,3])
print(k)

<class 'list'>


In [114]:
k=[1,2,3,4]
k.remove(1)
k

[2, 3, 4]